In [1]:
from langchain_community.chat_models import ChatOllama

#llm = ChatOllama(model="llama3.1:latest", request_timeout=60)
llm = ChatOllama(model="tinyllama:latest", request_timeout=60)

C:\Users\adria\AppData\Local\Temp\ipykernel_21500\2980094436.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="tinyllama:latest", request_timeout=60)


In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

loader = DirectoryLoader(
    path="./data/test",
    glob="**/*.txt",
    show_progress=True,
)

documents = loader.load()
print(f"Total characters: {len(documents[0].page_content)}")

C:\Users\adria\AppData\Local\Temp\ipykernel_21500\2586686258.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
c:\Users\adria\anaconda3\envs\LLM_ENV_LANGCHAIN\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 40%|████      | 2/5 [00:02<00:02,  1.04it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetyp

Total characters: 51546


In [3]:
print(documents[0].page_content[:500])

Biology is the scientific study of life. It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field. For instance, all organisms are composed of at least one cell that processes hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life. Energy processing is also important to life as it allows organisms to move, grow, and 


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(documents)

print(f"Split wiki pages into {len(all_splits)} sub-documents.")

Split wiki pages into 295 sub-documents.


In [5]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['185a30c4-50ec-4ab4-9e46-54c72ad421ac', 'a13c6a96-c708-40c1-9ef0-233d14da0438', '11b9ce9c-32bf-45e1-9d38-514f73797b38']


In [6]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {
        "answer": response.content,
        "context": state["context"]  # Include context in final state
    }


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [7]:
response = graph.invoke({"question": "What is cs?"})
print(response)

{'question': 'What is cs?', 'context': [Document(id='eb47a75b-a171-42d8-a6d7-542b5641aa18', metadata={'source': 'data\\test\\Computer_science.txt', 'start_index': 0}, page_content='Computer science is the study of computation, information, and automation. Computer science spans theoretical disciplines (such as algorithms, theory of computation, and information theory) to applied disciplines (including the design and implementation of hardware and software).'), Document(id='f4450350-39d0-4e5f-99ff-31c993ef408e', metadata={'source': 'data\\test\\Computer_science.txt', 'start_index': 14719}, page_content='Fields\n\nAs a discipline, computer science spans a range of topics from theoretical studies of algorithms and the limits of computation to the practical issues of implementing computing systems in hardware and software. CSAB, formerly called Computing Sciences Accreditation Board—which is made up of representatives of the Association for Computing Machinery (ACM), and the IEEE Computer 

In [8]:
print(response)

{'question': 'What is cs?', 'context': [Document(id='eb47a75b-a171-42d8-a6d7-542b5641aa18', metadata={'source': 'data\\test\\Computer_science.txt', 'start_index': 0}, page_content='Computer science is the study of computation, information, and automation. Computer science spans theoretical disciplines (such as algorithms, theory of computation, and information theory) to applied disciplines (including the design and implementation of hardware and software).'), Document(id='f4450350-39d0-4e5f-99ff-31c993ef408e', metadata={'source': 'data\\test\\Computer_science.txt', 'start_index': 14719}, page_content='Fields\n\nAs a discipline, computer science spans a range of topics from theoretical studies of algorithms and the limits of computation to the practical issues of implementing computing systems in hardware and software. CSAB, formerly called Computing Sciences Accreditation Board—which is made up of representatives of the Association for Computing Machinery (ACM), and the IEEE Computer 

In [9]:
for i, doc in enumerate(response["context"]):
    print(f"--- Document {i+1} ---")
    print(doc.page_content)
    print()


--- Document 1 ---
Computer science is the study of computation, information, and automation. Computer science spans theoretical disciplines (such as algorithms, theory of computation, and information theory) to applied disciplines (including the design and implementation of hardware and software).

--- Document 2 ---
Fields

As a discipline, computer science spans a range of topics from theoretical studies of algorithms and the limits of computation to the practical issues of implementing computing systems in hardware and software. CSAB, formerly called Computing Sciences Accreditation Board—which is made up of representatives of the Association for Computing Machinery (ACM), and the IEEE Computer Society (IEEE CS)—identifies four areas that it considers crucial to the discipline of computer science: theory of computation, algorithms and data structures, programming methodology and languages, and computer elements and architecture. In addition to these four areas, CSAB also identifies